In [1]:
from cdparser import Classifier, Features, LabeledEntry, Utils
import json
import ast
import numpy as np

In [3]:
## Create a classifier object and load some labeled data from a CSV
classifier = Classifier.Classifier()
classifier.load_training("1850_train_bk.csv")

In [4]:
## Optionally, load validation dataset
# classifier.load_validation("/full/path/to/validation/nypl-labeled-validate.csv")

## Train your classifier (with default settings)
classifier.train()

In [5]:
#read cleaned data
with open(r"BK_1850-51_cleaned.txt","r",encoding = "utf-8") as f:
    lines = f.readlines()

In [6]:
#remove newlines
lines = [x.strip() for x in lines] 

In [7]:
entry_dict = {}
for i in range(len(lines)):
## Create an entry object from string
    entry = LabeledEntry.LabeledEntry(lines[i])

    ## Pass the entry to the classifier
    classifier.label(entry)

    ## Export the labeled entry as JSON
    entry_dict[str(i+1)] = json.dumps(entry.categories)
# with open('CRF_output.txt', 'a') as fout:
#         json.dump(entry_dict,fout)

In [8]:
with open('entry_dict.txt', 'w',encoding="utf-8") as f:
        f.write(str(entry_dict))

In [9]:
entry_dict

{'1': '{"subjects": ["Abberly Richard"], "occupations": ["shoemaker"], "locations": [{"value": "168 Hudson av"}]}',
 '2': '{"subjects": ["Abberly Samuel"], "occupations": [], "locations": [{"value": "mason Dean n Vanderbilt av"}]}',
 '3': '{"subjects": ["Abbey A C"], "occupations": ["carpenter"], "locations": [{"value": "82 Poplar"}]}',
 '4': '{"subjects": ["Aessy Horatio G , Columbia Institute for the"], "occupations": [], "locations": [{"value": "education of"}]}',
 '5': '{"subjects": [], "occupations": [], "locations": [{"value": "boys 75 Columbia"}]}',
 '6': '{"subjects": ["Abbey WS"], "occupations": [], "locations": [{"value": "merchant N \\u00c2\\u00a5 h Union n Court"}]}',
 '7': '{"subjects": ["Abbott Abraham"], "occupations": ["marble sawyer"], "locations": [{"value": "Pacific n Boerum"}]}',
 '8': '{"subjects": ["Abbott Daniel"], "occupations": [], "locations": [{"value": "25 Front . [Court"}]}',
 '9': '{"subjects": ["Abbott Francis H"], "occupations": ["merchant"], "locations"

In [10]:

cities = ['Hoboken', 'N. J.', 'Spain', 'B’klyn', 'Mt. Vernon', 'Jersey City', 'Brooklyn', 'New J','WILLIAMSBURG', 'BUSHWICK', 'GREENPOINT',  'FLATBUSH', 'RAVENSWOOD','CAMPTOWN','CARMANSVLLE', 'WMSBURG','WULIAMSBURG', 'WMBURG','WMSBG','WMSBURGH', 'HOBOKEN', 'N. J.', 'S. ISLAND','SPAIN', 'B’KLYN', 'MT. VERNON','BROOKLYN','JERSEY CITY', 'N. JERSEY', 'CINCINNATI', 'GERMANY', 'STNTN', 'JERSEY C.', 'HOBOK', 'CONNECTICUT','BLYN',"JERSY CTY",'BRKLYN','NEW JERSEY','LONG ISLAND', 'N. ORLEANS', 'BKLYN','BROOKLYR','B.KLYN','JER. C.','CONN', 'L. I.', 'L.I.', 'L. I','L.I','L I','L I.', 'PTCHESTER', "B'KLYN",'J. C.','N.J.','S. I.','S.I.', 'NEW J']
def has_city(address, cities = cities): #returns city if present
    for city in cities:
#         print(city.upper(), [name.upper() for name in address.split()],city.upper() in [name.upper() for name in address])
        if city.capitalize() in address:
              return city
    return ''

In [11]:
full_entry =[]
for i in range(len(lines)):
    full_entry.append(str(i+1)+ ' '+lines[i])
# full_entry

In [13]:
address_list = ast.literal_eval(entry_dict[str(1)])['locations']
# address_list[1]

In [14]:
n = len(entry_dict)
final_output = {}
# generate fields
for i in range(1,n+1):
    cd = {}
    cd['Full Name'] = ''
    cd['First Name'] = ''
    cd['Last Name'] = ''
    cd['Middle Name'] = ''
    cd['Widow-flag'] = 'False'
    cd['Widow Of'] = ''
    cd['Title'] = ''
    cd['Occupation'] = ''
    cd['Full Address'] = ''
    cd['H Status-flag'] = ''
    cd['H Address'] = ''
    cd['H House Number'] = ''
    cd['H Street Name'] = ''
    cd['H City'] = ''
    cd['Address 2'] = ''
    cd['Address 2 House Number'] = ''
    cd['Address 2 Street Name'] = ''
    cd['Address 2 City'] = ''
#     full_entry_ls = full_entry[i-1].split()
#     cd['Index'] = full_entry_ls[0]
#     cd['Full Entry'] = ' '.join(full_entry_ls[1:])

    record = entry_dict[str(i)]
    # name
    try:
        fullname = ast.literal_eval(entry_dict[str(i)])['subjects'][0]
        fullname_list = fullname.split()
        cd['Full Name'] = fullname
        if(len(fullname_list) == 2):
            cd['Last Name'] = fullname_list[0]
            cd['First Name'] = fullname_list[1]
        elif(len(fullname_list) >= 3):
            cd['Last Name'] = fullname_list[0]
            cd['First Name'] = fullname_list[1]
            cd['Middle Name'] = fullname_list[2]
#         elif(fullname_list[2] == '.' or fullname_list[2] == ','):
#             cd['Last Name'] = fullname_list[0]
#             cd['First Name'] = fullname_list[1]
#             cd['Middle Name'] = fullname_list[3]
            
        
        if(len(ast.literal_eval(entry_dict[str(i)])['occupations'])):
            occupation_to_check_for_widows =  ast.literal_eval(entry_dict[str(i)])['occupations'][0]
            if('wid' in occupation_to_check_for_widows):
                cd['Widow-flag'] = 'True'
                cd['Widow Of'] = fullname_list[0]
                

    #widow flag
        if('wid' in ast.literal_eval(entry_dict[str(i)])['subjects'][-1]):
                cd['Widow-flag'] = 'True'
                cd['Widow Of'] = fullname_list[0]

    except:
        pass
  
  #occupation
    try:
        occupation_list =  ast.literal_eval(entry_dict[str(i)])['occupations']
        if(len(occupation_list) == 1):
            cd['Occupation'] = occupation_list[0]
        else:
            cd['Occupation'] = occupation_list[1]
    except:
        pass
  
#   #address
    try:
#         len(ast.literal_eval(entry_dict['3'])['locations'])
        address = ast.literal_eval(entry_dict[str(i)])['locations']
        full_add = ''
        for item in address:
            if('labels' in item.keys()):
                if('h' in item['labels']):
                    full_add += 'h. '
            full_add += item['value']
            full_add += ', '
        full_add = full_add[:-2]
        cd['Full Address'] = full_add
        if(len(address) == 1): #this means it is house address
            address_1 = address[0]['value'].split()
            cd['H Address'] = address[0]['value']
            if ('labels' in address[0].keys()):
                if('h' in address[0]['labels']):
                    cd['H Status-flag'] = 'True'
            else:
                cd['H Status-flag'] = 'Assume'
#                 start = 0 #index if string where address starts
#         elif(len(address) == 2):
#             if ('labels' in address[0].keys()):
#                 if('h' in address[0]['labels']):
#                     cd['H Status-flag'] = 'True'
#                     cd['H Address'] = address[0]['value']
#                     cd['H House Number'] = ''
#                     cd['H Street Name'] = ''
#                     cd['H City'] = ''
            house_num = False #finds starting of house nmber
            for item in address[0]['value'].split():
                if(item.isdigit()):
                    house_num = True
                    break
            if(house_num):
                cd['H House Number'] = item
        #city
                city = has_city(cities, address[0]['value'])
                if(city):
                    cd['H City'] = city
                    cd['H Street Name'] = ' '.join(address_1).replace(city,'').strip() #everythign except the city 
                else:
                    cd['H Street Name'] = ' '.join(address_1)
            else: #everything goes into city
                cd['H City'] = address_1

        else: #len of address >1, then 2nd add is house. 1st is other address.
            add = ast.literal_eval(entry_dict[str(i)])['locations'][0]['value']
            add_ls = add.split()
            cd['Address 2'] = add
            house_num = False
            if(add_ls[0].isdigit()):
                house_num = True
            if(house_num):
                cd['Address 2 House Number'] = add_ls[0]
            #city
                city = has_city(add)
                if(city):
                    cd['Address 2 City'] = city
                    cd['Address 2 Street Name'] = ' '.join(add_ls).replace(city, '').strip()
                else:
                    cd['Address 2 Street Name'] = ' '.join(add_ls)
            else:
                cd['Address 2 City'] = ' '.join(add_ls)
      
            add_1 =ast.literal_eval(entry_dict[str(i)])['locations'][1]['value']
            add_ls = add_1.split()
            if ('labels' in ast.literal_eval(entry_dict[str(140)])['locations'][1].keys()):
                if('h' in ast.literal_eval(entry_dict[str(140)])['locations'][1]['labels']):
                    cd['H Address'] = add_1
                    cd['H Status-flag'] = 'True'
            else:
                    cd['H Status-flag'] = 'Assume'

            house_num = False
            if(add_ls[0].isdigit()):
                house_num = True
            if(house_num):
                cd['H House Number'] = add_ls[0]
          #city
                city = has_city(add_1)
                if(city):
                    cd['H City'] = city
                    cd['H Street Name'] = ' '.join(add_ls).replace(city, '').strip()
                else:
                    cd['H Street Name'] = ' '.join(add_ls)
            else:
                cd['H City'] = ' '.join(add_ls)

    except:
        print('error in:', i+1)

    final_output[str(i)] = cd

error in: 88
error in: 151
error in: 156
error in: 166
error in: 205
error in: 207
error in: 272
error in: 287
error in: 408
error in: 487
error in: 521
error in: 590
error in: 732
error in: 779
error in: 875
error in: 915
error in: 974
error in: 1022
error in: 1062
error in: 1216
error in: 1350


In [15]:
final_output

{'1': {'Full Name': 'Abberly Richard',
  'First Name': 'Richard',
  'Last Name': 'Abberly',
  'Middle Name': '',
  'Widow-flag': 'False',
  'Widow Of': '',
  'Title': '',
  'Occupation': 'shoemaker',
  'Full Address': '168 Hudson av',
  'H Status-flag': 'Assume',
  'H Address': '168 Hudson av',
  'H House Number': '168',
  'H Street Name': '168 Hudson av',
  'H City': '',
  'Address 2': '',
  'Address 2 House Number': '',
  'Address 2 Street Name': '',
  'Address 2 City': ''},
 '2': {'Full Name': 'Abberly Samuel',
  'First Name': 'Samuel',
  'Last Name': 'Abberly',
  'Middle Name': '',
  'Widow-flag': 'False',
  'Widow Of': '',
  'Title': '',
  'Occupation': '',
  'Full Address': 'mason Dean n Vanderbilt av',
  'H Status-flag': 'Assume',
  'H Address': 'mason Dean n Vanderbilt av',
  'H House Number': '',
  'H Street Name': '',
  'H City': ['mason', 'Dean', 'n', 'Vanderbilt', 'av'],
  'Address 2': '',
  'Address 2 House Number': '',
  'Address 2 Street Name': '',
  'Address 2 City': ''

In [16]:
len(final_output)

1372

In [17]:
len(entry_dict)

1372

In [64]:
import pickle
# ...
with open('final_output_mn_1850.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(final_output, f, pickle.HIGHEST_PROTOCOL)

In [65]:
import pandas as pd

read_dict = pd.read_pickle('final_output_mn_1850.pickle')
read_dict

{'1': {'Full Name': 'Ackley Addison',
  'First Name': 'Addison',
  'Last Name': 'Ackley',
  'Middle Name': '',
  'Widow-flag': 'False',
  'Widow Of': '',
  'Title': '',
  'Occupation': 'clerk',
  'Full Address': '47 Broadway, h. 39 Pearl',
  'H Status-flag': 'True',
  'H Address': '39 Pearl',
  'H House Number': '39',
  'H Street Name': '39 Pearl',
  'H City': '',
  'Address 2': '47 Broadway',
  'Address 2 House Number': '47',
  'Address 2 Street Name': '47 Broadway',
  'Address 2 City': ''},
 '2': {'Full Name': 'Adams William',
  'First Name': 'William',
  'Last Name': 'Adams',
  'Middle Name': '',
  'Widow-flag': 'False',
  'Widow Of': '',
  'Title': '',
  'Occupation': 'weigher',
  'Full Address': '20 Washington & 13 West, h. 170 Clinton , Brooklyn',
  'H Status-flag': 'True',
  'H Address': '170 Clinton , Brooklyn',
  'H House Number': '170',
  'H Street Name': '170 Clinton ,',
  'H City': 'Brooklyn',
  'Address 2': '20 Washington & 13 West',
  'Address 2 House Number': '20',
  'Ad

In [18]:
import pandas as pd
output_1850_tess = pd.DataFrame.from_dict(final_output).transpose()

In [19]:
output_1850_tess.replace('', np.nan, inplace=True)
# output_1850_tess.sort_values(by=['Full Name'], inplace=True)

In [20]:
output_1850_tess = output_1850_tess.replace({'â€œ':''}, regex=True)
output_1850_tess= output_1850_tess.replace({'â€�':''}, regex=True)
output_1850_tess = output_1850_tess.replace({'â€”':''}, regex=True)
output_1850_tess.head(10)

,Full Name,First Name,Last Name,Middle Name,Widow-flag,Widow Of,Title,Occupation,Full Address,H Status-flag,H Address,H House Number,H Street Name,H City,Address 2,Address 2 House Number,Address 2 Street Name,Address 2 City
1,Abberly Richard,Richard,Abberly,NaN,False,NaN,NaN,shoemaker,168 Hudson av,Assume,168 Hudson av,168,168 Hudson av,NaN,NaN,NaN,NaN,NaN
2,Abberly Samuel,Samuel,Abberly,NaN,False,NaN,NaN,NaN,mason Dean n Vanderbilt av,Assume,mason Dean n Vanderbilt av,NaN,NaN,"[mason, Dean, n, Vanderbilt, av]",NaN,NaN,NaN,NaN
3,Abbey A C,A,Abbey,C,False,NaN,NaN,carpenter,82 Poplar,Assume,82 Poplar,82,82 Poplar,NaN,NaN,NaN,NaN,NaN
4,"Aessy Horatio G , Columbia Institute for the",Horatio,Aessy,G,False,NaN,NaN,NaN,education of,Assume,education of,NaN,NaN,"[education, of]",NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,boys 75 Columbia,Assume,boys 75 Columbia,75,boys 75 Columbia,NaN,NaN,NaN,NaN,NaN
6,Abbey WS,WS,Abbey,NaN,False,NaN,NaN,NaN,merchant N Â¥ h Union n Court,Assume,merchant N Â¥ h Union n Court,NaN,NaN,"[merchant, N, Â¥, h, Union, n, Court]",NaN,NaN,NaN,NaN
7,Abbott Abraham,Abraham,Abbott,NaN,False,NaN,NaN,marble sawyer,Pacific n Boerum,Assume,Pacific n Boerum,NaN,NaN,"[Pacific, n, Boerum]",NaN,NaN,NaN,NaN
8,Abbott Daniel,Daniel,Abbott,NaN,False,NaN,NaN,NaN,25 Front . [Court,Assume,25 Front . [Court,25,25 Front . [Court,NaN,NaN,NaN,NaN,NaN
9,Abbott Francis H,Francis,Abbott,H,False,NaN,NaN,merchant,141 Front N Â¥ h Joyalemonn,Assume,141 Front N Â¥ h Joyalemonn,141,141 Front N Â¥ h Joyalemonn,NaN,NaN,NaN,NaN,NaN
10,Abbott H B,H,Abbott,B,False,NaN,NaN,milkman,Kent ay north,Assume,Kent ay north,NaN,NaN,"[Kent, ay, north]",NaN,NaN,NaN,NaN


In [21]:
output_1850_tess.tail(10)

,Full Name,First Name,Last Name,Middle Name,Widow-flag,Widow Of,Title,Occupation,Full Address,H Status-flag,H Address,H House Number,H Street Name,H City,Address 2,Address 2 House Number,Address 2 Street Name,Address 2 City
1363,Bayer Edward,Edward,Bayer,NaN,False,NaN,NaN,NaN,280 Bridge n Myrtle av,Assume,280 Bridge n Myrtle av,280,280 Bridge n Myrtle av,NaN,NaN,NaN,NaN,NaN
1364,Bayley John,John,Bayley,NaN,False,NaN,NaN,contractor,96 Carl (108 Livingston,Assume,96 Carl (108 Livingston,96,96 Carl (108 Livingston,NaN,NaN,NaN,NaN,NaN
1365,Baylis Abraham B,Abraham,Baylis,B,False,NaN,NaN,broker,30 Merchants Exchange N YÂ¥ h,Assume,30 Merchants Exchange N YÂ¥ h,30,30 Merchants Exchange N YÂ¥ h,NaN,NaN,NaN,NaN,NaN
1366,Baylis Catharine,Catharine,Baylis,NaN,False,NaN,NaN,boarding house,193 Pearl,Assume,193 Pearl,193,193 Pearl,NaN,NaN,NaN,NaN,NaN
1367,Baylis David B,David,Baylis,B,False,NaN,NaN,hardware merchant,134 Atlantic,Assume,134 Atlantic,134,134 Atlantic,NaN,NaN,NaN,NaN,NaN
1368,Baylis Daniel,Daniel,Baylis,NaN,False,NaN,NaN,spar maker,131 Hudson av,Assume,131 Hudson av,131,131 Hudson av,NaN,NaN,NaN,NaN,NaN
1369,Baylis Gilbert C,Gilbert,Baylis,C,False,NaN,NaN,huxter,Washington market N YÂ¥ h 101 Jay,Assume,Washington market N YÂ¥ h 101 Jay,101,Washington market N YÂ¥ h 101 Jay,NaN,NaN,NaN,NaN,NaN
1370,Baylis Thos,Thos,Baylis,NaN,False,NaN,NaN,builder,83 Washington h Joralemoan Henry,Assume,83 Washington h Joralemoan Henry,83,83 Washington h Joralemoan Henry,NaN,NaN,NaN,NaN,NaN
1371,Baylis Thomas,Thomas,Baylis,NaN,False,NaN,NaN,NaN,gardener Clove road Bedford,Assume,gardener Clove road Bedford,NaN,NaN,"[gardener, Clove, road, Bedford]",NaN,NaN,NaN,NaN
1372,Beach Alfred E,Alfred,Beach,E,False,NaN,NaN,editor,73 Columbia,Assume,73 Columbia,73,73 Columbia,NaN,NaN,NaN,NaN,NaN


In [22]:
output_1850_tess = output_1850_tess.loc[output_1850_tess.astype(str).drop_duplicates().index]
len(output_1850_tess)

1350

In [23]:
#filter those with name and no address
output_1850_tess = output_1850_tess[~output_1850_tess['Full Name'].isna()]

In [24]:
len(output_1850_tess)

1329

In [25]:
len(output_1850_tess)

1329

In [26]:
output_1850_tess = output_1850_tess[~output_1850_tess['Full Address'].isna()]

In [27]:
len(output_1850_tess)

1308

In [28]:
output_1850_tess_fn = output_1850_tess[output_1850_tess['Full Name'].str.len() > 1]

In [29]:
len(output_1850_tess_fn)

1308

In [123]:
output_1850_tess = output_1850_tess[~output_1850_tess['Full Address'].isna()]
len(output_1850_tess)

76197

In [30]:
output_1850_tess.to_csv('output_1850_tess.csv')

In [97]:
len(output_1850_tess)

109336

In [99]:
output_1850_tess.head(15)

,Full Name,First Name,Last Name,Middle Name,Widow-flag,Widow Of,Title,Occupation,Full Address,H Status-flag,H Address,H House Number,H Street Name,H City,Address 2,Address 2 House Number,Address 2 Street Name,Address 2 City
73577,' - . Fulton,-,',.,False,NaN,NaN,NaN,h. Williamsburg on,True,Williamsburg on,NaN,NaN,"[Williamsburg, on]",NaN,NaN,NaN,NaN
98603,' . E,.,',E,False,NaN,NaN,NaN,. Broadway,Assume,. Broadway,NaN,NaN,"[., Broadway]",NaN,NaN,NaN,NaN
70758,' Bridge Paulina W,Bridge,',Paulina,True,',NaN,widow of Lewis K,40 E . 14th,Assume,40 E . 14th,40,40 E . 14th,NaN,NaN,NaN,NaN,NaN
8294,' Dudy John,Dudy,',John,False,NaN,NaN,shoemaker,3 Sixth,Assume,3 Sixth,3,3 Sixth,NaN,NaN,NaN,NaN,NaN
40993,' Dudy John,Dudy,',John,False,NaN,NaN,laborer,"3 Sixth Duffy John, 114 Mulberry",True,114 Mulberry,114,114 Mulberry,NaN,3 Sixth Duffy John,3,3 Sixth Duffy John,NaN
10667,' FARMERS' LOAN & TRUST CO,FARMERS',',LOAN,False,NaN,NaN,NaN,28 Exchange pl . |,Assume,28 Exchange pl . |,28,28 Exchange pl . |,NaN,NaN,NaN,NaN,NaN
43357,' FARMERS' LOAN & TRUST CO . 28 Exchange pl . ...,FARMERS',',LOAN,False,NaN,NaN,laborer,200 Second,Assume,200 Second,200,200 Second,NaN,NaN,NaN,NaN,NaN
43371,' Farnham P . J . & Co,Farnham,',P,False,NaN,NaN,laborer,"85 Pearl Farrell William, 491 Av . 6",True,491 Av . 6,491,491 Av . 6,NaN,85 Pearl Farrell William,85,85 Pearl Farrell William,NaN
10692,' Farnham P . J . & Co,Farnham,',P,False,NaN,NaN,importers,85 Pearl,Assume,85 Pearl,85,85 Pearl,NaN,NaN,NaN,NaN,NaN
55464,' Hollings Martin,Hollings,',Martin,False,NaN,NaN,grocer,125 Franklia,Assume,125 Franklia,125,125 Franklia,NaN,NaN,NaN,NaN,NaN


In [103]:
output_1850_tess = output_1850_tess[output_1850_tess['Full Name'].str.len() > 3]

In [111]:
len(output_1850_tess)

75644

In [104]:
output_1850_tess.head(15)

,Full Name,First Name,Last Name,Middle Name,Widow-flag,Widow Of,Title,Occupation,Full Address,H Status-flag,H Address,H House Number,H Street Name,H City,Address 2,Address 2 House Number,Address 2 Street Name,Address 2 City
73577,' - . Fulton,-,',.,False,NaN,NaN,NaN,h. Williamsburg on,True,Williamsburg on,NaN,NaN,"[Williamsburg, on]",NaN,NaN,NaN,NaN
98603,' . E,.,',E,False,NaN,NaN,NaN,. Broadway,Assume,. Broadway,NaN,NaN,"[., Broadway]",NaN,NaN,NaN,NaN
70758,' Bridge Paulina W,Bridge,',Paulina,True,',NaN,widow of Lewis K,40 E . 14th,Assume,40 E . 14th,40,40 E . 14th,NaN,NaN,NaN,NaN,NaN
8294,' Dudy John,Dudy,',John,False,NaN,NaN,shoemaker,3 Sixth,Assume,3 Sixth,3,3 Sixth,NaN,NaN,NaN,NaN,NaN
40993,' Dudy John,Dudy,',John,False,NaN,NaN,laborer,"3 Sixth Duffy John, 114 Mulberry",True,114 Mulberry,114,114 Mulberry,NaN,3 Sixth Duffy John,3,3 Sixth Duffy John,NaN
10667,' FARMERS' LOAN & TRUST CO,FARMERS',',LOAN,False,NaN,NaN,NaN,28 Exchange pl . |,Assume,28 Exchange pl . |,28,28 Exchange pl . |,NaN,NaN,NaN,NaN,NaN
43357,' FARMERS' LOAN & TRUST CO . 28 Exchange pl . ...,FARMERS',',LOAN,False,NaN,NaN,laborer,200 Second,Assume,200 Second,200,200 Second,NaN,NaN,NaN,NaN,NaN
43371,' Farnham P . J . & Co,Farnham,',P,False,NaN,NaN,laborer,"85 Pearl Farrell William, 491 Av . 6",True,491 Av . 6,491,491 Av . 6,NaN,85 Pearl Farrell William,85,85 Pearl Farrell William,NaN
10692,' Farnham P . J . & Co,Farnham,',P,False,NaN,NaN,importers,85 Pearl,Assume,85 Pearl,85,85 Pearl,NaN,NaN,NaN,NaN,NaN
55464,' Hollings Martin,Hollings,',Martin,False,NaN,NaN,grocer,125 Franklia,Assume,125 Franklia,125,125 Franklia,NaN,NaN,NaN,NaN,NaN


In [37]:
#names that are not 2 or three words long
n = len(entry_dict)
count = 0
namelengths =[]
for i in range(1,n+1):
    try:
        name = ast.literal_eval(entry_dict[str(i)])['subjects'][0]
        name = name.split()
        if(len(name) != 2 and len(name) != 3):
            print(name, len(name))
            count += 1
            namelengths.append(len(name))
    except:
        pass
print(count)
print(namelengths)

['Allen', 'E', '.', 'W'] 4
['Barrowclough', 'James', ',', 'E'] 4
['Amerman', 'Geo', '.', 'V'] 4
['Amerman', 'O', '.', 'V', '.', '(Rev'] 6
['Amerman', 'Wm', '.', 'C'] 4
['Amswick', 'L', '.', 'E'] 4
['Anderson', 'Chas', '.', 'F'] 4
['Anderson', 'Eliza', 'J', '.', '(col’d)', 'wid', '.', 'Thos'] 8
['Anderson', 'Jas', '.', 'J'] 4
['Anderson', 'James', 'W', '.', 'Broadway'] 5
['Anderson', 'John', '&', 'Co'] 4
['Anderson', 'John', 'H', '.', '&', 'Co'] 6
['Anderson', 'Robert', 'S', '.', 'N'] 5
['Anderson', 'Wm', '.', 'Theodore'] 4
['Andersén', 'C', '.', 'V', '.', '&', 'J'] 7
['Anderson', ',', 'Ritter', '&', 'Co'] 5
['ANDERSON', ',', 'STARR', '&', 'CO', '.'] 6
['Anderton', 'Ralph', '.', 'L'] 4
['Andrews', 'F', '.', 'A'] 4
['Andrews', 'Loring', '&', 'Co'] 4
['Andrews', 'Robt', '.', 'J'] 4
['Andrews', '§', '.', 'B'] 4
['Andrews', 'Thomas', 'M', '.', '15', 'Cortlandt'] 6
['Andrews', 'Wm', '.', 'D'] 4
['Andrews', 'Wm', '.', 'M'] 4
['Andrews', 'B', '.', '&', 'Co'] 5
['ANGEVINE', 'DANIEL', '&', 'LEVI

In [57]:
'Wid' in ast.literal_eval(entry_dict['103'])['occupations'][0]

False

In [41]:
entry_dict['113536']

'{"subjects": [], "occupations": [], "locations": [{"value": "34th n , Av . 9"}]}'

In [45]:
full_entry[113535]

'113536 34th n, Av. 9'